In [1]:
from adult_read import *

Epoch: 1 | Batch index: 0 | Batch size: 128
break minibatch for-loop
Epoch: 2 | Batch index: 0 | Batch size: 128
break minibatch for-loop


In [2]:
# logistic regression: very slow
def FedAvg(clients, pred_var, sen_var = None, learning_rate = .01, w_init = np.ones(len(adult.columns)-1), seed = 123, server_itr = 100,
          clients_itr = 20, batch_size = 20, threshold = 10**(-8)):
    
    np.random.seed(seed)
    
    def logLikelihood(w, client):
        l = 0
        var = client.drop(pred_var, axis = 1)
        pred = client[pred_var]
        for i in client.index:
            exp = np.exp(w.dot(var.loc[i]))
            l += np.log((exp/(1+exp))**pred.loc[i]* (1/(1+exp))**(1-pred.loc[i]))
        return l
    
    def gradient(w, client, batch_idx):
        grad = 0
        var = client.drop(pred_var, axis = 1)
        pred = client[pred_var]
        for i in batch_idx:
            exp = min(np.exp(-w.dot(var.loc[i])), 10**10)
            coef = pred[i] - 1/(1+exp)
            grad += coef * var.loc[i]
        return grad
    
    def client_update(w, client):
        for i in range(clients_itr):
            batch_idx = np.random.choice(client.index, batch_size)
            w_new = w + learning_rate * gradient(w, client, batch_idx)
            if sum((w_new - w)**2) <= threshold:
                break
            else:
                w = w_new
        return w
    
    # server
    w = w_init
    for i in range(server_itr):
        print("step %d -----" % i)
        weights = np.zeros(len(clients[0].columns)-1)
        for client in clients:
            weights += client_update(w, client)
        w = weights/len(clients)
        print("log likelihood: %f" % logLikelihood(w, adult))
    # loss
    return w

In [3]:
FedAvg(clients, 'salary')

step 0 -----


KeyboardInterrupt: 